#### **Import**

In [29]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

#### **Data Descriptions:**

* Marque - Marque: Brand of the product
* Produit - Acheteur Mode vs Design: Product category split by responsible buyer (Mode vs Design).
* Produit - Nom Produit: Product name.
* SKU - Référence: Unique product reference (SKU).
* Valeur du stock utilisable (hors bundle) HT: Monetary value of usable stock (excluding bundle products), expressed hors taxes (HT).
* CAMV net HT: Net Cost of Goods Sold (COGS), hors taxes.
* Nb de jours avec du stock utilisable: Number of days during which the product had usable stock available.

In [31]:
data = pd.read_csv('data.csv')

# show the data
data.head()

,Marque - Marque,Produit - Acheteur Mode vs Design,Produit - Nom Produit,SKU - Référence,Valeur du stock utilisable (hors bundle) HT,CAMV net HT,Nb de jours avec du stock utilisable
0,Stone Island,mode,Sweat Capuche Logo,AAA1590769,-6474.0,-581.00,56.0
1,Bonton,mode,Cardigan Lilou Cœurs Pointelle,AAA1586863,-4968.0,0.00,55.0
2,Sisters Department,mode,Sweatshirt Julie Coton Bio,AAA1568398,-3733.0,-74.66,56.0
3,American Vintage,mode,Echarpe Vitow Laine et Alpaga,AAA1567986,-3588.0,-676.00,45.0
4,Petit Nord,mode,Chaussons Fourrées Shearling Bootie,AAA1544679,-3168.0,0.00,55.0


In [32]:
# show all columns
data.columns

Index(['Marque - Marque', 'Produit - Acheteur Mode vs Design',
       'Produit - Nom Produit', 'SKU - Référence',
       'Valeur du stock utilisable (hors bundle) HT', 'CAMV net HT',
       'Nb de jours avec du stock utilisable'],
      dtype='object')

In [33]:
# rename columns
columns = {
    x: y for x, y in zip(data.columns, ['marque', 'mode_design', 'name', 'sku', 'total_stock_value', 'total_cogs', 'days_stock'])
}

data.rename(columns=columns, inplace=True)

data.head()

,marque,mode_design,name,sku,total_stock_value,total_cogs,days_stock
0,Stone Island,mode,Sweat Capuche Logo,AAA1590769,-6474.0,-581.00,56.0
1,Bonton,mode,Cardigan Lilou Cœurs Pointelle,AAA1586863,-4968.0,0.00,55.0
2,Sisters Department,mode,Sweatshirt Julie Coton Bio,AAA1568398,-3733.0,-74.66,56.0
3,American Vintage,mode,Echarpe Vitow Laine et Alpaga,AAA1567986,-3588.0,-676.00,45.0
4,Petit Nord,mode,Chaussons Fourrées Shearling Bootie,AAA1544679,-3168.0,0.00,55.0


**Before diving into the analysis, It is important to check the datatypes to avoid error during calculation, merge, etc.**

In [34]:
# look at the data types
data.dtypes

marque                object
mode_design           object
name                  object
sku                   object
total_stock_value     object
total_cogs           float64
days_stock           float64
dtype: object

In [35]:
data['total_stock_value'].values

array([-6474.0, -4968.0, -3733.0, ..., '238655', '1094594.49',
       '1518180.2'], shape=(406690,), dtype=object)

**We found that *total_stock_value* is the type object, so we need to change it, we found that it was written in french notation, which consider commas as point.**

In [36]:
data['total_stock_value'] = pd.to_numeric(
    data['total_stock_value']
    .astype(str)
    .str.replace(',', '', regex=False),  # remove commas
    errors='coerce'                      # invalid types with become NaN instead of error
)

# check datatypes
data.dtypes

marque                object
mode_design           object
name                  object
sku                   object
total_stock_value    float64
total_cogs           float64
days_stock           float64
dtype: object

In [37]:
def coverage(D10, E10, C10):
    
    # Cap value equivalent to 365 * 2.5
    '''
    D10: Usable stock value - represent the monetary value of the available stock
    E10: Cost of goods sold (negative by default) - used as the consumption rate
    C10: Number of day with usable stock

    The function estimates how many days the current stock can cover given the consumption rate.

    Daily consumption = -E10/C10
    Coverage_days = D10 / Daily consumption = (D10 / -E10) * C10
    The value of coverage is capped at 365*2.5, which is 2.5 years

    Returns: coverage_days
    
    '''
    cap = 365 * 2.5

    if D10 > 0:
        if E10 < 0:
            result = min(D10 / (-E10) * C10, cap)
        else:
            result = cap
    else:
        result = 0

    return result


def classify_fsn(coverage, day_stock):
    """
    coverage: Estimated number of days the current stock can cover
    day_stock: number of days the product has been available in stock

    Return: FSN categories (New, Fast, Medium, Non-moving, Slow)
    """

    if day_stock <= 90:
        return 'new'
    else:
        if coverage <= 90:
            return "fast_90"
        elif coverage <= 180:
            return "medium_365"
        elif coverage > 912:  # > 2.5 ans
            return "non_moving"
        else:
            return "slow_912"


In [38]:
# apply the function for each row in the data

data['coverage'] = data.apply(
    lambda row: coverage(
        row['total_stock_value'],
        row['total_cogs'],
        row['days_stock']
    ),
    axis=1 
)

data.head()

,marque,mode_design,name,sku,total_stock_value,total_cogs,days_stock,coverage
0,Stone Island,mode,Sweat Capuche Logo,AAA1590769,-6474.0,-581.00,56.0,0.0
1,Bonton,mode,Cardigan Lilou Cœurs Pointelle,AAA1586863,-4968.0,0.00,55.0,0.0
2,Sisters Department,mode,Sweatshirt Julie Coton Bio,AAA1568398,-3733.0,-74.66,56.0,0.0
3,American Vintage,mode,Echarpe Vitow Laine et Alpaga,AAA1567986,-3588.0,-676.00,45.0,0.0
4,Petit Nord,mode,Chaussons Fourrées Shearling Bootie,AAA1544679,-3168.0,0.00,55.0,0.0


In [39]:
# classify each SKU into the FSN categories

data['fsn'] = data.apply(
    lambda row: classify_fsn(row["coverage"], row["days_stock"]),
    axis=1
)

data.head()

,marque,mode_design,name,sku,total_stock_value,total_cogs,days_stock,coverage,fsn
0,Stone Island,mode,Sweat Capuche Logo,AAA1590769,-6474.0,-581.00,56.0,0.0,new
1,Bonton,mode,Cardigan Lilou Cœurs Pointelle,AAA1586863,-4968.0,0.00,55.0,0.0,new
2,Sisters Department,mode,Sweatshirt Julie Coton Bio,AAA1568398,-3733.0,-74.66,56.0,0.0,new
3,American Vintage,mode,Echarpe Vitow Laine et Alpaga,AAA1567986,-3588.0,-676.00,45.0,0.0,new
4,Petit Nord,mode,Chaussons Fourrées Shearling Bootie,AAA1544679,-3168.0,0.00,55.0,0.0,new


In [40]:
# group data per brand and mode design

data_per_brand = (
    data.groupby(['marque', 'mode_design']).agg({
        'total_stock_value': 'sum',
        'total_cogs': 'sum',
    })
)

data_per_brand.head()

,,total_stock_value,total_cogs
marque,mode_design,,
& Tradition,design,29530.355,-52955.01
&Klevering,design,21690.650,-42216.95
1+ in the family,mode,276584.600,-350924.80
10.03.53,mode,2266.000,-2106.00
23heures59 éditions,design,637.000,-143.00


In [41]:
# group data per brand, mode_design, and fsn categories

data_per_fsn = data.groupby(['marque', 'mode_design', 'fsn']).agg({
    'total_stock_value': 'sum'
}).reset_index()

pivot_table = data_per_fsn.pivot(index=['marque', 'mode_design'], columns='fsn', values='total_stock_value').fillna(0) 


pivot_table.head()


,fsn,fast_90,medium_365,new,non_moving,slow_912
marque,mode_design,,,,,
& Tradition,design,-145.18,4164.13,130.24,10680.375,14700.79
&Klevering,design,2865.80,4887.80,4420.65,1651.300,7865.10
1+ in the family,mode,18396.40,32350.90,-1873.90,115850.300,111860.90
10.03.53,mode,0.00,0.00,0.00,1792.000,474.00
23heures59 éditions,design,0.00,0.00,0.00,624.000,13.00


In [42]:
# join the two dataframe

fsn_data = data_per_brand.join(pivot_table).reset_index()
fsn_data.head()

,marque,mode_design,total_stock_value,total_cogs,fast_90,medium_365,new,non_moving,slow_912
0,& Tradition,design,29530.355,-52955.01,-145.18,4164.13,130.24,10680.375,14700.79
1,&Klevering,design,21690.650,-42216.95,2865.80,4887.80,4420.65,1651.300,7865.10
2,1+ in the family,mode,276584.600,-350924.80,18396.40,32350.90,-1873.90,115850.300,111860.90
3,10.03.53,mode,2266.000,-2106.00,0.00,0.00,0.00,1792.000,474.00
4,23heures59 éditions,design,637.000,-143.00,0.00,0.00,0.00,624.000,13.00


In [44]:
# export as CSV file

fsn_data.to_csv('fsn_data.csv', index=False)

In [45]:
liewood = data[data['marque']=='Liewood']
liewood.head()

,marque,mode_design,name,sku,total_stock_value,total_cogs,days_stock,coverage,fsn
36,Liewood,mode,Tote Bag Big,AAA1572616,-840.0,-56.0,48.0,0.0,new
94,Liewood,mode,Tote Bag Big,AAA1572615,-386.4,-56.0,54.0,0.0,new
200,Liewood,design,Set vaisselle Brody en PLA - set de 4,AAA1021907,-182.0,0.0,0.0,0.0,new
250,Liewood,design,Parure de lit en coton bio,AAA1220810,-144.0,0.0,0.0,0.0,new
251,Liewood,design,Tapis d'éveil Glenn en coton bio,AAA0741206,-144.0,0.0,0.0,0.0,new


In [46]:
# for brand liewood only

liewood = data[data['marque']=='Liewood']
liewood.head()

,marque,mode_design,name,sku,total_stock_value,total_cogs,days_stock,coverage,fsn
36,Liewood,mode,Tote Bag Big,AAA1572616,-840.0,-56.0,48.0,0.0,new
94,Liewood,mode,Tote Bag Big,AAA1572615,-386.4,-56.0,54.0,0.0,new
200,Liewood,design,Set vaisselle Brody en PLA - set de 4,AAA1021907,-182.0,0.0,0.0,0.0,new
250,Liewood,design,Parure de lit en coton bio,AAA1220810,-144.0,0.0,0.0,0.0,new
251,Liewood,design,Tapis d'éveil Glenn en coton bio,AAA0741206,-144.0,0.0,0.0,0.0,new


In [47]:
# top 5 for each FSN categories for the brand liewood only

top5 = (
    liewood.groupby('fsn', group_keys=False)
        .apply(lambda df: df.sort_values(by='total_stock_value', ascending=False).head(5))
        .reset_index(drop=True)
)

top5.head()

,marque,mode_design,name,sku,total_stock_value,total_cogs,days_stock,coverage,fsn
0,Liewood,design,Calendrier de l'avent Babbo,AAA1530421,958.80,-3419.72,241.0,67.570093,fast_90
1,Liewood,mode,Veste Nelson Jacket Polyester Recyclé - Colle...,AAA1254794,864.00,-2256.00,132.0,50.553191,fast_90
2,Liewood,design,Rangement Damien,AAA1426903,512.70,-1641.14,283.0,88.410556,fast_90
3,Liewood,design,Ardoise magique Venzora,AAA1690420,496.32,-586.56,100.0,84.615385,fast_90
4,Liewood,design,Ardoise magique Zora,AAA1494960,493.23,-818.04,136.0,82.000000,fast_90


In [48]:
# export as CSV

top5.drop(columns=['coverage', 'days_stock']).to_csv('top5_liewood.csv', index=False)